In [ ]:
%%writefile finetune_phi3_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./src

command: >- 
  echo "python finetune_phi3.py ${{inputs.data_dir}} ${{outputs.out_dir}}"; sleep 60000
inputs:
  data_dir: 
    type: uri_folder
    #mode: ro_mount
    #path: azureml:burbery_data@latest
    path: azureml:Sujet-Finance-Vision-10k@latest
outputs:
  out_dir: 
    type: custom_model
    mode: upload
environment: azureml:llava_finetuning:16
environment_variables:
    WANDB_MODE: disabled
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 1
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
#compute: azureml:fine-tune-cluster
compute: azureml:a100-low-priority
display_name: finetune_phi3_vision_job_5
#name: finetune_phi3_vision_job
experiment_name: finetune_phi3_vision
description: Finetune Phi3 Vision


In [ ]:
!az ml job create -f finetune_phi3_job.yaml

In [ ]:
%%writefile endpoint_phi3_env_jo.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
command: >- 
  echo "Model Dir: ${{inputs.model_dir}}"; sleep 6000
inputs:
  model_dir: 
    type: custom_model
    path: azureml:finetuned_phi3_vision@latest
environment: azureml:llava_finetuning_inference:2
environment_variables:
    WANDB_MODE: disabled
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 1
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
compute: azureml:fine-tune-cluster
#compute: azureml:a100-low-priority
display_name: endpoint_phi3_env_job
#name: endpoint_phi3_env_job
experiment_name: finetune_phi3_vision
description: Endpoint Phi3 Vision


In [ ]:
!az ml job create -f endpoint_phi3_env_jo.yaml

In [ ]:
%%writefile ./endpoint/endpoint.yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: finetunedPhi3Endpoint
auth_mode: key

In [7]:
!az ml online-endpoint create --file ./endpoint/endpoint.yaml

{
  "auth_mode": "key",
  "id": "/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/onlineEndpoints/finetunedphi3endpoint",
  "identity": {
    "principal_id": "c0258f71-08d2-4e8c-aa35-dbdb2f483ced",
    "tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "type": "system_assigned"
  },
  "kind": "Managed",
  "location": "northeurope",
  "mirror_traffic": {},
  "name": "finetunedphi3endpoint",
  "openapi_uri": "https://finetunedphi3endpoint.northeurope.inference.ml.azure.com/swagger.json",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/providers/Microsoft.MachineLearningServices/locations/northeurope/mfeOperationsStatus/oeidp:23317781-110c-4faf-9763-19f2fd34b6cf:d2b3ec37-40db-43d3-93b6-a4bc75ffe32b?api-version=2022-02-01-preview",
    "azureml.onlineendpointid": "/subscriptions/781b03e7-6eb7-4506-bab8-

In [ ]:
%%writefile ./endpoint/deployment.yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: finetunedPhi3Endpoint
model: azureml:finetuned_phi3_vision:1
code_configuration:
  code: .
  scoring_script: score.py
environment: azureml:llava_finetuning_inference:2
instance_type: Standard_NC12s_v3
instance_count: 1

In [8]:
!az ml online-deployment create --all-traffic --file ./endpoint/deployment.yaml

..................................................................................................................................................{
  "app_insights_enabled": false,
  "code_configuration": {
    "code": "/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/ffd4e7ed-77bc-4a35-a524-cd57d9aee2a3/versions/1",
    "scoring_script": "score.py"
  },
  "egress_public_network_access": "enabled",
  "endpoint_name": "finetunedphi3endpoint",
  "environment": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/environments/llava_finetuning_inference/versions/2",
  "environment_variables": {
    "AML_APP_ROOT": "/var/azureml-app/endpoint",
    "AZUREML_ENTRY_SCRIPT": "score.py",
    "AZUREML_MODEL_DIR": "/var/azureml-app/azureml-models/finetuned_phi3_vision/1"
  },
  "id": 

All traffic will be set to deployment blue once it has been provisioned.
If you interrupt this command or it times out while waiting for the provisioning, you can try to set all the traffic to this deployment later once its has been provisioned.
Check: endpoint finetunedPhi3Endpoint exists
